In [1]:
## Importando a Biblioteca
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [2]:
## Carregando o dataset
data=pd.read_csv("C:\Mestrado\Diabetes\Dataset\diabetes_prediction_dataset.csv")
data.head()

<>:2: SyntaxWarning: invalid escape sequence '\M'
<>:2: SyntaxWarning: invalid escape sequence '\M'
C:\Users\lucas\AppData\Local\Temp\ipykernel_18880\3264958230.py:2: SyntaxWarning: invalid escape sequence '\M'
  data=pd.read_csv("C:\Mestrado\Diabetes\Dataset\diabetes_prediction_dataset.csv")


,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [3]:
## Agora vamos codificar as variáveis categóricas
label_encoder_genderdi=LabelEncoder()
data['gender']=label_encoder_genderdi.fit_transform(data['gender'])
data

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,0,80.0,0,1,never,25.19,6.6,140,0
1,0,54.0,0,0,No Info,27.32,6.6,80,0
2,1,28.0,0,0,never,27.32,5.7,158,0
3,0,36.0,0,0,current,23.45,5.0,155,0
4,1,76.0,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
99995,0,80.0,0,0,No Info,27.32,6.2,90,0
99996,0,2.0,0,0,No Info,17.37,6.5,100,0
99997,1,66.0,0,0,former,27.83,5.7,155,0
99998,0,24.0,0,0,never,35.42,4.0,100,0


In [6]:
## Se utilizarmos o encoder na feature geography teremos valores como 0, 1 e 2. Nesse caso o computador vai atribuir
## hierarquia para essas categorias, o que não tem. Por isso para essa feature será utilizado o One hot encode
from sklearn.preprocessing import OneHotEncoder
## Criando a variável e a instância para o one hot encoder
onehot_encoder_smoking=OneHotEncoder()
smoking_encoder=onehot_encoder_smoking.fit_transform(data[['smoking_history']])
smoking_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 100000 stored elements and shape (100000, 6)>

In [7]:
## Verificando os nomes dentro da feature geography
onehot_encoder_smoking.get_feature_names_out(['smoking_history'])

array(['smoking_history_No Info', 'smoking_history_current',
       'smoking_history_ever', 'smoking_history_former',
       'smoking_history_never', 'smoking_history_not current'],
      dtype=object)

In [8]:
## Fazendo um dataframe para essa feature codificada
smoking_encoded_df=pd.DataFrame(smoking_encoder.toarray(),columns=onehot_encoder_smoking.get_feature_names_out(['smoking_history']))
## Aqui o geo_encoder foi transformado em matriz porque haverá erro no tamanho se usar ele normal
## o número de colunas é igual ao número de nomes que tem dentro da feature geography
smoking_encoded_df

,smoking_history_No Info,smoking_history_current,smoking_history_ever,smoking_history_former,smoking_history_never,smoking_history_not current
0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
99995,1.0,0.0,0.0,0.0,0.0,0.0
99996,1.0,0.0,0.0,0.0,0.0,0.0
99997,0.0,0.0,0.0,1.0,0.0,0.0
99998,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
## Combinando todas as colunas com os dados
## Estamos retirando a coluna geography e adicionando a codificada que criamos
data=pd.concat([data.drop('smoking_history',axis=1),smoking_encoded_df],axis=1)
data.head()

,gender,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,diabetes,smoking_history_No Info,smoking_history_current,smoking_history_ever,smoking_history_former,smoking_history_never,smoking_history_not current
0,0,80.0,0,1,25.19,6.6,140,0,0.0,0.0,0.0,0.0,1.0,0.0
1,0,54.0,0,0,27.32,6.6,80,0,1.0,0.0,0.0,0.0,0.0,0.0
2,1,28.0,0,0,27.32,5.7,158,0,0.0,0.0,0.0,0.0,1.0,0.0
3,0,36.0,0,0,23.45,5.0,155,0,0.0,1.0,0.0,0.0,0.0,0.0
4,1,76.0,1,1,20.14,4.8,155,0,0.0,1.0,0.0,0.0,0.0,0.0


In [10]:
## Como o label encoder e o one hot encoder serão utilizados depois, devemos salvar o que já utilizamos
## no formato de arquivo pickle
## Salvando os encoders e scaler
with open('label_encoder_genderdi.pkl','wb') as file:
    pickle.dump(label_encoder_genderdi,file)

with open('onehot_encoder_smoking.pkl','wb') as file:
    pickle.dump(onehot_encoder_smoking,file)

In [13]:
## Dividindo a base de dados em features dependentes e independentes
X=data.drop('diabetes',axis=1)
y=data['diabetes']

## Dividindo em dados de teste e de treino
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

## Scale os features
scalerdi=StandardScaler()
X_train=scalerdi.fit_transform(X_train)
X_test=scalerdi.transform(X_test)

In [12]:
X_train

array([[ 1.18538964e+00,  1.37871976e+00, -2.86309234e-01, ...,
         3.10684138e+00, -7.35110211e-01, -2.62931169e-01],
       [ 1.18538964e+00,  1.68986587e+00, -2.86309234e-01, ...,
        -3.21870310e-01,  1.36034024e+00, -2.62931169e-01],
       [-8.42217184e-01, -1.77010751e-01, -2.86309234e-01, ...,
        -3.21870310e-01, -7.35110211e-01, -2.62931169e-01],
       ...,
       [ 1.18538964e+00,  7.87021842e-04, -2.86309234e-01, ...,
        -3.21870310e-01,  1.36034024e+00, -2.62931169e-01],
       [-8.42217184e-01, -2.21460194e-01, -2.86309234e-01, ...,
        -3.21870310e-01,  1.36034024e+00, -2.62931169e-01],
       [-8.42217184e-01, -8.43752400e-01, -2.86309234e-01, ...,
        -3.21870310e-01,  1.36034024e+00, -2.62931169e-01]],
      shape=(80000, 13))

In [14]:
## Salvando o scaler no formato pickle
with open('scalerdi.pkl','wb') as file:
    pickle.dump(scalerdi,file)

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [16]:
## Construindo o modelo ANN
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),   ## HL1 conectada ao input
    Dense(32,activation='relu'),   ## HL2
    Dense(1,activation='sigmoid')  ## Output Layer
]


)

c:\Mestrado\env\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [17]:
## Com o Summary pode-se verificar quantos parâmetros tem nessa ANN
## Os parâmetros são os weights e o bias
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,009 (11.75 KB)

 Trainable params: 3,009 (11.75 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
import tensorflow
## Se colocar direto o optimizer direto no compile, ele irá utilizar o learning rate de default do optimizer
## Para poder fazer a escolha do learning rate pode-se colocar o optimizer em uma variável
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
## Acima foi definido o Optimizer Adam com um learning rate de 0.01
## Pode-se fazer a mesma coisa para o loss
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss

<LossFunctionWrapper(<function binary_crossentropy at 0x0000014AF47D4680>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>

In [19]:
## Agora será feito o compile do modelo
## Pode-se escrever dessa forma e colocar o optimizer dentro do código, mas dessa forma
## não poderá ser definido o learning rate
## model.compile(optimizer="Adam",loss="binary_crossentropy",metric=['accuracy'])

## Por isso será feito da seguinte forma
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [20]:
## Preparando o Tensorboard
## Para o Tensorboard serão necessárias algumas bibliotecas
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
## Aqui serão guardados os logs

## Para o Tensorboard tem-se
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [21]:
## Definindo o Early Stopping
## Ele serve para parar a ANN se não houver mais alteração nas perdas. Em outras palavras
## Se for definida 1000 Epochs para a ANN rodar, mas o erro já fica muito baixo com 800
## O computador irá parar de rodar a ANN em 800 e não rodará até 1000
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)
## monitor - serve para monitorar um parâmetro
## patience - Se não houver mudança dentro de 5 Epochs, ele para
## restore_best_weights - Ele pode pegar o melhor valor do weights de uma epoch anterior

In [22]:
## Treinando o modelo
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.9622 - loss: 0.1078 - val_accuracy: 0.9660 - val_loss: 0.0948
Epoch 2/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9674 - loss: 0.0928 - val_accuracy: 0.9670 - val_loss: 0.0936
Epoch 3/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9687 - loss: 0.0897 - val_accuracy: 0.9704 - val_loss: 0.0881
Epoch 4/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9693 - loss: 0.0882 - val_accuracy: 0.9706 - val_loss: 0.0872
Epoch 5/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9696 - loss: 0.0875 - val_accuracy: 0.9705 - val_loss: 0.0856
Epoch 6/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9699 - loss: 0.0869 - val_accuracy: 0.9697 - val_loss: 0.0860
Epoch 7/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9706 - loss: 0.0861 - val_accuracy: 0.9679 - val_loss: 0.0911
Epoch 8/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9706 - loss: 0

In [23]:
## Agora o modelo será salvo no formato h5, que é compatível com o Keras
model.save('modeldi.h5')

In [24]:
## Carregando uma extensão do tensorboard
%load_ext tensorboard

In [26]:
## Com o tensorboard pode-se verificar os logs
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 19288), started 0:00:20 ago. (Use '!kill 19288' to kill it.)